# Securité avancé

## Encryption homomorphique 

Installer la librairie si besoin.

In [1]:
!pip install tenseal

Librairie utilisées.

In [2]:
import random
import math
import numpy as np
from sympy.ntheory.residue_ntheory import primitive_root
import tenseal as ts
import time
import sys

Quelques fonctions globales, utiles.

In [3]:
# Fonction du TP de crypthographie
def algo_Euclide(a, b) : 
    ri_moins_1 = a
    ri = b
    si_moins_1 = 1
    si = 0
    ti_moins_1 = 0
    ti = 1
    qi = a // b
    
    ri_plus_1 = 1

    while (ri_plus_1 != 0) :
        ri_plus_1 = ri_moins_1 - (qi * ri)
        si_plus_1 = si_moins_1 - (qi * si)
        ti_plus_1 = ti_moins_1 - (qi * ti)
        if (ri_plus_1 != 0) : # on decale nos valeurs si on continue de boucler (0 -> -1, +1 -> 0)
            qi = ri // ri_plus_1
            
            ri_moins_1 = ri
            ri = ri_plus_1
            
            si_moins_1 = si
            si = si_plus_1
            
            ti_moins_1 = ti
            ti = ti_plus_1

    return ri, si, ti

# Fonction du TP de crypthographie
def expo_rapide(n, puissance, mod) :
    nombre = bin(puissance)[2:] # on transforme la puissance en binaire pour savoir jusqu'a quelle puissance on doit faire l'expo rapide
    nb = len(nombre)
    liste_puiss = [0 for x in range(nb)]
    liste_puiss[0] = n
    for b in range(1, nb) : # on calcule la valeur pour chaque puissance de 2
        liste_puiss[b] = pow(liste_puiss[b - 1], 2) % mod
    total = 1
    for p in range(0, nb) : # on multiplie chaque valeur correpondante a notre tableau en appliquant le modulo a chaque fois
        if (nombre[p] == '1') :
            total = (total * liste_puiss[nb - p - 1]) % mod
    return total

# Vérifier si un nombre est premier (Miller-Rabin simplifié)
def is_prime(n, k=20) :
    if n < 2 :
        return False
    for _ in range(k) :
        a = random.randint(2, n)
        if expo_rapide(a, n - 1, n) != 1 :
            return False
    return True

# Facteurs premiers
def prime_factors(n) :
    factors = set()
    d = 2
    while d * d <= n :
        while n % d == 0 :
            factors.add(d)
            n //= d
        d += 1
    if n > 1 :
        factors.add(n)
    return factors

### RSA cryptosystem

In [4]:
# Génération des clés
def keygen(bits=512) :
    # Génération de deux grands nombres premiers p et q
    p = random.getrandbits(bits)
    q = random.getrandbits(bits)
    while not (is_prime(p) and is_prime(q) and p != q) :
        p = random.getrandbits(bits)
        q = random.getrandbits(bits)
    
    n = p * q
    lambd = np.lcm(p-1, q-1) # Plus petit multiple commun
    
    e = random.randrange(2, lambd)
    while not (is_prime(e) and math.gcd(e, lambd) == 1) :
        e = random.randrange(2, lambd)
     
    r, s, t = algo_Euclide(lambd, e)
    if r != 1 :
        raise ValueError(f"{e} n'a pas d'inverse mod {lambd}")
        
    d = t % lambd
    
    return (n, e), (n, d)  # Clé publique, clé privée

# Chiffrement d'un message m
def encrypt(m, pub_key) :
    n, e = pub_key
    cipher = expo_rapide(m, e, n)
    return cipher

# Déchiffrement d'un message chiffré
def decrypt(c, priv_key) :
    n, d = priv_key
    m = expo_rapide(c, d, n)
    return m

# Multiplication homomorphe
def multiply_homomorphic(c1, c2, n) :
    return (c1*c2) % n


# TEST
pub_key, priv_key = keygen(256)  # Utilisation de 256 bits pour rapidité/complexité (en réalité, 512+ bits)
n, e = pub_key

m1, m2 = 5, 3
c1 = encrypt(m1, pub_key)
c2 = encrypt(m2, pub_key)

print(f"Message 1 : {m1}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m1)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c1)} octets")

print(f"Message 2 : {m2}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m2)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c2)} octets")

start_mult = time.time()
mult = m1 * m2
end_mult = time.time()
print("\nTemps pour faire la multiplication normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication : {mult}")
print(f"Taille du nombre normal multiplié : {sys.getsizeof(mult)} octets")

start_mult = time.time()
c_mult = multiply_homomorphic(c1, c2, n)
decrypted_mult = decrypt(c_mult, priv_key)
end_mult = time.time()
print("\nTemps pour faire la multiplication encrypté : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication homomorphe (après déchiffrement) : {decrypted_mult}")
print(f"Taille du nombre encrypté multiplié : {sys.getsizeof(c_mult)} octets")


Message 1 : 5
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 92 octets
Message 2 : 3
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 92 octets

Temps pour faire la multiplication normal :  0.0 ms
Résultat après multiplication : 15
Taille du nombre normal multiplié : 28 octets

Temps pour faire la multiplication encrypté :  1.0013580322265625 ms
Résultat après multiplication homomorphe (après déchiffrement) : 15
Taille du nombre encrypté multiplié : 92 octets


### ElGamal encryption

In [5]:
# Génération des clés
def keygen(bits=512) :
    # Génération de deux grands nombres premiers p et q
    p = random.getrandbits(bits)
    while not (is_prime(p)) :
        p = random.getrandbits(bits)
    
    q = p - 1
    g = primitive_root(p)
    
    x = random.randrange(1, q)
    
    h = expo_rapide(g, x, p)
    
    
    return (p, q, g, h), x  # Clé publique, clé privée

# Chiffrement d'un message m
def encrypt(M, pub_key) :
    p, q, g, h = pub_key

    # Mapping M vers m ∈ Z*_p
    m = M % p
    if m == 0 :
        raise ValueError("Le message ne peut pas être 0 dans Z*_p")

    y =  random.randrange(1, q)
    s = expo_rapide(h, y, p)

    c1 = expo_rapide(g, y, p)
    c2 = (m * s) % p

    return (c1, c2)

# Déchiffrement d'un message chiffré
def decrypt(c, priv_key, pub_key) :
    c1, c2 = c
    p, q, g, h = pub_key
    x = priv_key

    s = expo_rapide(c1, x, p)
    s_inv = expo_rapide(c1, q - x, p)
    m = (c2 * s_inv) % p

    return m

# Multiplication homomorphe
def multiply_homomorphic(c1, c2) :
    c_11, c_12 = c1
    c_21, c_22 = c2
    return (c_11 * c_21, c_12 * c_22)


# TEST
pub_key, priv_key = keygen(256)  # Utilisation de 256 bits pour rapidité/complexité (en réalité, 512+ bits)
p, q, g, h = pub_key

m1, m2 = 5, 3
c1 = encrypt(m1, pub_key)
c2 = encrypt(m2, pub_key)

print(f"Message 1 : {m1}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m1)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c1)} octets")

print(f"Message 2 : {m2}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m2)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c2)} octets")

start_mult = time.time()
mult = m1 * m2
end_mult = time.time()
print("\nTemps pour faire la multiplication normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication : {mult}")
print(f"Taille du nombre normal multiplié : {sys.getsizeof(mult)} octets")

start_mult = time.time()
c_mult = multiply_homomorphic(c1, c2)
decrypted_mult = decrypt(c_mult, priv_key, pub_key)
end_mult = time.time()
print("\nTemps pour faire la multiplication encrypté : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication homomorphe (après déchiffrement) : {decrypted_mult}")
print(f"Taille du nombre encrypté multiplié : {sys.getsizeof(c_mult)} octets")


Message 1 : 5
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 56 octets
Message 2 : 3
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 56 octets

Temps pour faire la multiplication normal :  0.0 ms
Résultat après multiplication : 15
Taille du nombre normal multiplié : 28 octets

Temps pour faire la multiplication encrypté :  0.9996891021728516 ms
Résultat après multiplication homomorphe (après déchiffrement) : 15
Taille du nombre encrypté multiplié : 56 octets


### Goldwasser–Micali cryptosystem

In [6]:
# Génération des clés
def legendre_symbol(a, p):
    return expo_rapide(a, (p - 1) // 2, p) if a % p != 0 else 0

def keygen(bits=512) :
    # Génération de deux grands nombres premiers p et q
    p = random.getrandbits(bits)
    q = random.getrandbits(bits)
    while not (is_prime(p) and is_prime(q) and p != q) :
        p = random.getrandbits(bits)
        q = random.getrandbits(bits)
    
    N = p * q
    
    x = random.randrange(2, N)
    while not (legendre_symbol(x, p) == p - 1 and legendre_symbol(x, q) == q - 1) :
        x = random.randrange(2, N)
    
    return (x, N), (p, q)  # Clé publique, clé privée

# Transforme un int en liste de bits
def int_to_bits(n, bit_length=8) :
    return [int(b) for b in bin(n)[2:].zfill(bit_length)]

# Chiffrement d'un message m
def encrypt(m, pub_key) :
    mess = int_to_bits(m, num_bits)
    x, N = pub_key
    ciphertext = []

    for bit in mess :
        # Générer y tel que gcd(y, N) = 1
        y = random.randint(2, N - 1)
        while math.gcd(y, N) != 1 :
            y = random.randint(2, N - 1)

        c = (expo_rapide(y, 2, N) * expo_rapide(x, bit, N)) % N
        ciphertext.append(c)
    return ciphertext

# Transforme une liste de bits en int
def bits_to_int(bits) :
    return int("".join(map(str, bits)), 2)

# Déchiffrement d'un message chiffré
def decrypt(c, priv_key) :
    p, q = priv_key
    mess = []
    
    for cipher in c :
        bit = -1
        if legendre_symbol(cipher, p) == 1 and legendre_symbol(cipher, q) == 1 :
            bit = 0
        else :
            bit = 1
        mess.append(bit)
          
    m = bits_to_int(mess)
    return m

# Multiplication homomorphe
def add_bit_homomorphic(c1, c2, n) :
    res = []
    for b1, b2 in zip(c1,c2) :
        val = (b1 * b2) % n
        res.append(val)
    return res


# TEST
num_bits = 256
pub_key, priv_key = keygen(num_bits)  # Utilisation de 256 bits pour rapidité/complexité (en réalité, 512+ bits)
x, N = pub_key

m1, m2 = 5, 3
c1 = encrypt(m1, pub_key)
c2 = encrypt(m2, pub_key)

print(f"Message 1 : {m1}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m1)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c1)} octets")

print(f"Message 2 : {m2}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m2)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c2)} octets")

start_mult = time.time()
xor = m1 ^ m2
end_mult = time.time()
print("\nTemps pour faire le xor normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après xor : {xor}")
print(f"Taille du nombre normal après xor : {sys.getsizeof(xor)} octets")

start_mult = time.time()
c_mult = add_bit_homomorphic(c1, c2, N)
decrypted_mult = decrypt(c_mult, priv_key)
end_mult = time.time()
print("\nTemps pour faire le xor encrypté : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après xor homomorphe (après déchiffrement) : {decrypted_mult}")
print(f"Taille du nombre encrypté après xor : {sys.getsizeof(c_mult)} octets")


Message 1 : 5
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 2208 octets
Message 2 : 3
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 2208 octets

Temps pour faire le xor normal :  0.0 ms
Résultat après xor : 6
Taille du nombre normal après xor : 28 octets

Temps pour faire le xor encrypté :  228.99603843688965 ms
Résultat après xor homomorphe (après déchiffrement) : 6
Taille du nombre encrypté après xor : 2208 octets


### Benaloh cryptosystem

In [7]:
# Génération des clés
def keygen(r, bits=512) :
    # Génération de deux grands nombres premiers p et q
    k = random.getrandbits(bits - r.bit_length())  # Réduit le champ
    p = r * k + 1
    q = random.getrandbits(bits)
    while not (is_prime(p) and math.gcd(r, k) == 1) :
        k = random.getrandbits(bits - r.bit_length())  # Réduit le champ
        p = r * k + 1
    while not (is_prime(q) and math.gcd(r, q - 1) == 1) :
        q = random.getrandbits(bits)

    n = p * q
    phi = (p - 1) * (q - 1)

    r_factors = prime_factors(r)
    y = random.randrange(2, n)

    while True :
        y = random.randrange(2, n)
        if math.gcd(y, n) != 1 :
            continue # Reset si pas inversible
        valid = True
        for pi in r_factors :
            if expo_rapide(y, phi // pi, n) == 1 :
                valid = False
                break # Arrête la boucle for
        if valid :
            break # Sort du while

    x = expo_rapide(y, phi // r, n)
    
    return (y, n), (phi, x)  # Clé publique, clé privée

# Chiffrement d'un message m
def encrypt(m, pub_key, r) :
    y, n = pub_key
    
    if not (0 <= m < r) :
        raise ValueError(f"Le message m={m} doit être dans Z_r (0 ≤ m < {r})")
        
    u = random.randrange(2, n)
    while not math.gcd(u, n) == 1 : # u inversible mod n
        u = random.randrange(2, n)
        
    c = (expo_rapide(y, m, n) * expo_rapide(u, r, n)) % n
    return c

# Déchiffrement d'un message chiffré
def decrypt(c, priv_key, pub_key, r) :
    phi, x = priv_key
    y, n = pub_key

    a = expo_rapide(c, phi // r, n)

    for m in range(r) :
        if expo_rapide(x, m, n) == a :
            return m
        
    raise ValueError("Impossible de trouver m, le logarithme discret a échoué")
    
# Addition homomorphe modulo r
def add_homomorphic(c1, c2) :
    return c1 * c2


# TEST
r = random.getrandbits(32)
print(f"r = {r}")
pub_key, priv_key = keygen(r, 256)  # Utilisation de 256 bits pour rapidité/complexité (en réalité, 512+ bits)
y, n = pub_key

m1, m2 = 5, 3
c1 = encrypt(m1, pub_key, r)
c2 = encrypt(m2, pub_key, r)

print(f"Message 1 : {m1}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m1)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c1)} octets")

print(f"Message 2 : {m2}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m2)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c2)} octets")

start_mult = time.time()
add = m1 + m2
end_mult = time.time()
print("\nTemps pour faire l'addition normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après addition : {add}")
print(f"Taille du nombre normal additionné : {sys.getsizeof(add)} octets")

start_sum = time.time()
c_sum = add_homomorphic(c1, c2)
decrypted_sum = decrypt(c_sum, priv_key, pub_key, r)
end_sum = time.time()
print("\nTemps pour faire la somme encrypté : ", (end_sum-start_sum)*1000, "ms")
print(f"Résultat après somme homomorphe modulo {r} (après déchiffrement) : {decrypted_sum}")
print(f"Taille du nombre encrypté additionné : {sys.getsizeof(c_sum)} octets")

r = 1320055277
Message 1 : 5
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 92 octets
Message 2 : 3
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 92 octets

Temps pour faire l'addition normal :  0.0 ms
Résultat après addition : 8
Taille du nombre normal additionné : 28 octets

Temps pour faire la somme encrypté :  0.9987354278564453 ms
Résultat après somme homomorphe modulo 1320055277 (après déchiffrement) : 8
Taille du nombre encrypté additionné : 160 octets


### Paillier cryptosystem

In [8]:
# Génération des clés
def keygen(bits=512) :
    # Génération de deux grands nombres premiers p et q
    p = random.getrandbits(bits)
    q = random.getrandbits(bits)
    while not (is_prime(p) and is_prime(q) and math.gcd(p*q, (p-1)*(q-1)) == 1) :
        p = random.getrandbits(bits)
        q = random.getrandbits(bits)
    
    n = p * q
    lambd = np.lcm(p-1, q-1) # Plus petit multiple commun
    
    n_carre = n**2
    g = random.randrange(2, n_carre)  # On évite 1 pour des raisons de sécurité
    while math.gcd(g, n_carre) != 1 :
        g = random.randrange(2, n_carre)
    
    
    arg = expo_rapide(g, lambd, n_carre)
    L = (arg - 1) // n # Division entière
    r, s, t = algo_Euclide(L, n)
    if r != 1 :
        raise ValueError(f"{L} n'a pas d'inverse mod {n}")
    mu = s % n
    
    return (n, g), (lambd, mu)  # Clé publique, clé privée

# Chiffrement d'un message m
def encrypt(m, pub_key) :
    n, g = pub_key
    r = random.randrange(1, n)  # Générer un aléa
    while math.gcd(r, n) != 1 :
        r = random.randrange(1, n)
      
    n_carre = n**2
    cipher = (expo_rapide(g, m, n_carre) * expo_rapide(r, n, n_carre)) % (n_carre)
    return cipher

# Déchiffrement d'un message chiffré
def decrypt(c, priv_key, pub_key) :
    n, g = pub_key
    lambd, mu = priv_key
    arg = expo_rapide(c, lambd, n**2)
    L = (arg - 1) // n
    return (L * mu) % n

# Addition homomorphe
def add_homomorphic(c1, c2, n) :
    return (c1 * c2) % (n**2)

# Multiplication homomorphe
def multiply_homomorphic(c1, m2, n) :
    return expo_rapide(c1, m2, n**2)

# TEST
pub_key, priv_key = keygen(256)  # Utilisation de 256 bits pour rapidité/complexité (en réalité, 512+ bits)
n, g = pub_key

m1, m2 = 5, 3
c1 = encrypt(m1, pub_key)
c2 = encrypt(m2, pub_key)

print(f"Message 1 : {m1}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m1)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c1)} octets")

print(f"Message 2 : {m2}")
print(f"\tTaille du nombre normal : {sys.getsizeof(m2)} octets")
print(f"\tTaille du nombre encrypté : {sys.getsizeof(c2)} octets")

start_mult = time.time()
add = m1 + m2
end_mult = time.time()
print("\nTemps pour faire l'addition normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après addition : {add}")
print(f"Taille du nombre normal additionné : {sys.getsizeof(add)} octets")

start_mult = time.time()
mult = m1 * m2
end_mult = time.time()
print("\nTemps pour faire la multiplication normal : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication : {mult}")
print(f"Taille du nombre normal multiplié : {sys.getsizeof(mult)} octets")

start_sum = time.time()
c_sum = add_homomorphic(c1, c2, n)
decrypted_sum = decrypt(c_sum, priv_key, pub_key)
end_sum = time.time()
print("\nTemps pour faire la somme encrypté : ", (end_sum-start_sum)*1000, "ms")
print(f"Résultat après somme homomorphe (après déchiffrement) : {decrypted_sum}")
print(f"Taille du nombre encrypté additionné : {sys.getsizeof(c_sum)} octets")

start_mult = time.time()
c_mult= multiply_homomorphic(c1, m2, n)
decrypted_mult = decrypt(c_mult, priv_key, pub_key)
end_mult = time.time()
print("\nTemps pour faire la multiplication encrypté : ", (end_mult-start_mult)*1000, "ms")
print(f"Résultat après multiplication homomorphe (après déchiffrement) : {decrypted_mult}")
print(f"Taille du nombre encrypté multiplié : {sys.getsizeof(c_mult)} octets")


Message 1 : 5
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 160 octets
Message 2 : 3
	Taille du nombre normal : 28 octets
	Taille du nombre encrypté : 160 octets

Temps pour faire l'addition normal :  0.0 ms
Résultat après addition : 8
Taille du nombre normal additionné : 28 octets

Temps pour faire la multiplication normal :  0.0 ms
Résultat après multiplication : 15
Taille du nombre normal multiplié : 28 octets

Temps pour faire la somme encrypté :  3.999948501586914 ms
Résultat après somme homomorphe (après déchiffrement) : 8
Taille du nombre encrypté additionné : 160 octets

Temps pour faire la multiplication encrypté :  4.000186920166016 ms
Résultat après multiplication homomorphe (après déchiffrement) : 15
Taille du nombre encrypté multiplié : 160 octets


### Encryption homomorphe plus complexe avec la librairie TenSEAL

In [13]:
# Initialisation du contexte de chiffrement BFV
context = ts.context(
    ts.SCHEME_TYPE.BFV, 
    poly_modulus_degree=8192, 
    plain_modulus=786433,  
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.generate_galois_keys()
context.generate_relin_keys()

# Chiffrement des nombres
num1, num2 = 5, 3

# Temps pour les opérations normales (non chiffrées)
start_plain = time.time()
sum_plain = num1 + num2
end_plain = time.time()
sum_time = end_plain - start_plain

start_plain = time.time()
prod_plain = num1 * num2
end_plain = time.time()
mult_time = end_plain - start_plain

# Chiffrement
encrypted_num1 = ts.bfv_vector(context, [num1])
encrypted_num2 = ts.bfv_vector(context, [num2])

# Temps pour les opérations chiffrées
start_encrypted = time.time()
encrypted_sum = encrypted_num1 + encrypted_num2  # Addition homomorphe
end_encrypted = time.time()
encrypted_time_sum = end_encrypted - start_encrypted

start_encrypted = time.time()
encrypted_prod = encrypted_num1 * encrypted_num2  # Multiplication homomorphe
end_encrypted = time.time()
encrypted_time_mult = end_encrypted - start_encrypted


# Déchiffrement
sum_decrypted = encrypted_sum.decrypt()
prod_decrypted = encrypted_prod.decrypt()

# Affichage des résultats
print(f"Addition (non chiffrée) : {sum_plain} | Temps : {sum_time*1000:.8f} ms")
print(f"Addition (chiffrée) : {sum_decrypted[0]} | Temps : {encrypted_time_sum*1000:.8f} ms")

print(f"\nMultiplication (non chiffrée) : {prod_plain} | Temps : {mult_time*1000:.8f} ms")
print(f"Multiplication (chiffrée) : {prod_decrypted[0]} | Temps : {encrypted_time_mult*1000:.8f} ms")


Addition (non chiffrée) : 8 | Temps : 0.00000000 ms
Addition (chiffrée) : 8 | Temps : 0.45084953 ms

Multiplication (non chiffrée) : 15 | Temps : 0.00000000 ms
Multiplication (chiffrée) : 15 | Temps : 29.84118462 ms


In [14]:
# Taille des nombres normaux
print(f"Taille d'un entier normal : {sys.getsizeof(num1)} octets")

# Taille des nombres chiffrés
print(f"Taille d'un entier chiffré : {sys.getsizeof(encrypted_num1)} octets")  # Attention : sys.getsizeof() ne marche pas directement pour TenSEAL


Taille d'un entier normal : 28 octets
Taille d'un entier chiffré : 48 octets


In [15]:
encrypted_serialized = encrypted_num1.serialize()
print(f"Taille d'un entier chiffré (sérialisé) : {len(encrypted_serialized)} octets")


Taille d'un entier chiffré (sérialisé) : 334405 octets
